<a href="https://www.kaggle.com/code/wassimchouchen/best-model-for-ner?scriptVersionId=97788001" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [1]:
import numpy as np
import pandas as pd 
import seaborn as sns
import matplotlib.pyplot as plt 
import matplotlib.style as stl 
from tensorflow import keras 
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
import string
from tensorflow.keras.models import Sequential,Model
from tensorflow.keras.layers import Dense, LSTM, Embedding,Dropout,SpatialDropout1D,Conv1D,MaxPooling1D,GRU,BatchNormalization
import warnings
warnings.filterwarnings('ignore')
stl.use('ggplot')
from tqdm import tqdm
import string
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer

In [2]:
data=pd.read_csv("../input/sample/sample-2.csv")

# Preprocessing

In [3]:
text=data["transcription_text"]

In [4]:
stop_words = set(stopwords.words('english'))
punctuation = string.punctuation
stemmer = SnowballStemmer(language="english")
lemmatizer = WordNetLemmatizer()

In [5]:
stop_words.remove("of")
punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
def process(s):
    for p in punctuation:
        s = s.replace(p, '')
    s = word_tokenize(s)
    s = [w for w in s if not w in stop_words] #optional
    s = [lemmatizer.lemmatize(word) for word in s]
    return s

In [7]:
for i in tqdm(range(data.shape[0])):
    text[i] = process(text[i])

100%|██████████| 20/20 [00:02<00:00,  9.03it/s]


In [8]:
for i in tqdm(range(data.shape[0])):
    text[i] = " ".join(text[i])

100%|██████████| 20/20 [00:00<00:00, 2781.46it/s]


In [9]:
text

0     8 E encl I Anne H Elliott widow solemnly swear...
1     I E C Whaley solemnly swear I carefully read A...
2     43M I Mrs E Montgomery solemnly Swear I carefu...
3     No 2324 I solemnly swear affirm presence of Al...
4     August 24 1865 APPLICATION AND CERTIFICATE OF ...
5     I Wm Whaley solemnly swear I carefully read Am...
6     I J F Steinmeyer solemnly swear I carefully re...
7     I Joseph Edings solemnly swear I carefully rea...
8     Vicksburg Miss July 22 1867 Thomas Andrew Oath...
9     455 RETAINED COPY I Jacob Schroder solemnly sw...
10    image eagle AMNESTY OATH Mississippi City Miss...
11    OATH I P P Bergevin solemnly swear I never vol...
12                                         Frank Conner
13    I Caroline Belitzer solemnly Swear I carefully...
14    Triplicate AMNESTY OATH Pass Christian Miss Au...
15    Retained Copy HEADQUARTERS UNITED STATES FORCE...
16                                      Robert A Wilson
17    APPLICATION AND CERTIFICATE OF James D Ken

# spacy

In [10]:
import spacy
from spacy import displacy
from collections import Counter
import en_core_web_sm
nlp = en_core_web_sm.load()

In [11]:
L1=[]
def NER (txt):
    for i in range(20):
        nlp_text=nlp(txt[i])
        L1.append([(word.text, word.start_char, word.end_char, word.label_,spacy.explain(word.label_)) for word in nlp_text.ents])
        
NER(text)
l0=[]
l1=[]
l2=[]
l3=[]
l4=[]
l5=[]
for i in range(len(L1)):
    for j in range(len(L1[i])):
        l0.append(L1[i][j][0])
        l2.append(L1[i][j][1])
        l3.append(L1[i][j][2])
        l4.append(L1[i][j][3])
        l5.append(L1[i][j][4])

df1=pd.DataFrame({"entities":l0,"start_char":l2,"end_char":l3 ,"label": l4, "explain": l5, "frequency":""})
df1.head()

,entities,start_char,end_char,label,explain,frequency
0,Anne H Elliott,11,25,PERSON,"People, including fictional",
1,Amnesty,54,61,ORG,"Companies, agencies, institutions, etc.",
2,Andrew Johnson,82,96,PERSON,"People, including fictional",
3,United States of America,110,134,GPE,"Countries, cities, states",
4,May 29th 1865,135,148,DATE,Absolute or relative dates or periods,


In [12]:
Person=[]
ORG=[]
DATE=[]
GPE=[]

for i in range(df1.shape[0]):
    if df1.label.loc[i]=="PERSON":
        Person.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="ORG" :
        ORG.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="DATE" :
        DATE.append(df1.entities.loc[i])
    elif df1.label.loc[i]=="GPE" :
        GPE.append(df1.entities.loc[i])
GPE=np.array(GPE)
ORG=np.array(ORG)
Person=np.array(Person)
DATE=np.array(DATE)

In [13]:
GPE

array(['United States of America', 'South Carolina', 'Charleston',
       'United States of America', 'Charleston', 'Charleston',
       'United States of America', 'Charleston',
       'No1984 United States of America', 'Charleston', 'New Orleans',
       'US', 'United States of America', 'Charleston', 'Charleston',
       'United States of America', 'Charleston',
       'United States of America', 'United States of America',
       'United States of America', 'United States of America',
       'South Carolina', 'Vicksburg', 'United States of America',
       'South Carolina', 'Mississippi City', 'United States', 'States',
       'Mississippi City', 'Mississippi City', 'United States',
       'United States', 'United States', 'United States', 'United States',
       'Mississippi', 'United States of America', 'Charleston',
       'United States of America', 'United States', 'States',
       'United States', 'Gainsville', 'NC', 'United States', 'States',
       'United States'], dtype='

In [14]:
Person

array(['Anne H Elliott', 'Andrew Johnson',
       'Anne H Elliott United States of America', 'Anne H Elliott',
       'Hazel', 'Whaley', 'Andrew Johnson', 'Strikethrough', 'Oaths',
       'Oaths', 'Hu Kennedy', 'Amnesty Oath Administered At', 'Hiserodt',
       'William Whaley Sworn', 'William Whaley Sworn', 'Brown',
       'Charleston Sc', 'County', 'JF Steinmeyer Sworn', 'Charleston Sc',
       'Gry', 'Oaths', 'Joseph Edings', 'Thomas Andrew Oath of',
       'Jacob Schroder', 'Jacob Schroder Sworn', 'Walhalla', 'Walhalla',
       'Laborer', 'Amnesty Oath', 'Woodville Wilkinson', 'Frank Conner',
       'Copy F K Field Capt', 'Triplicate AMNESTY', 'Hiram Smith',
       'Hiram Smith', 'Pass Christian', 'Captn', 'Amnesty Oath',
       'Hiram Smith', 'Captn Commanding TRIPLICATE', 'Retained Copy',
       'Bolt Capt 54th', 'Robert A',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Gibbs Henderson', 'City Miss', 'Amnesty Oath', 'Frank E Miller'],
      dtype='<

# Hugging face  : Flair

In [15]:
!pip install transformers
!pip install flair

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 401.9/401.9 kB 7.3 MB/s eta 0:00:00
  Installing build dependencies ... - \ | / done
  Getting requirements to build wheel ... - \ | / - done
  Preparing metadata (pyproject.toml) ... - \ | / - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 32.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.1/53.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 44.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 788.5/788.5 kB 36.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.3/46.3 kB 2.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Preparing metadata (setup.py) ... - done
  Created wheel for gdown: filename=gdown-4.4.0-py3-none-

**ner-english-ontonotes-large** : the best one on this type of text

In [16]:
from flair.data import Sentence
from flair.models import SequenceTagger
T2=[]
D={}

tagger = SequenceTagger.load("flair/ner-english-ontonotes-large")

for i in range(20):
        sentencess =Sentence(text[i])
        tagger.predict(sentencess)
        for entity in sentencess.get_spans('ner'):
                     T2.append(entity)
                     if (entity.get_label("ner").score > 0.98) & (entity.get_label("ner").value != "CARDINAL") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "WORK_OF_ART") & (entity.get_label("ner").value != "LAW")& (entity.get_label("ner").value != "QUANTITY") & (entity.get_label("ner").value != "EVENT")  :
                         D[entity.text]= entity.get_label("ner").value

Downloading:   0%|          | 0.00/2.24G [00:00<?, ?B/s]

2022-06-08 07:50:56,199 loading file /root/.flair/models/ner-english-ontonotes-large/2da6c2cdd76e59113033adf670340bfd820f0301ae2e39204d67ba2dc276cc28.ec1bdb304b6c66111532c3b1fc6e522460ae73f1901848a4d0362cdf9760edb1


Downloading:   0%|          | 0.00/616 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/8.68M [00:00<?, ?B/s]

2022-06-08 07:51:14,563 SequenceTagger predicts: Dictionary with 76 tags: <unk>, O, B-CARDINAL, E-CARDINAL, S-PERSON, S-CARDINAL, S-PRODUCT, B-PRODUCT, I-PRODUCT, E-PRODUCT, B-WORK_OF_ART, I-WORK_OF_ART, E-WORK_OF_ART, B-PERSON, E-PERSON, S-GPE, B-DATE, I-DATE, E-DATE, S-ORDINAL, S-LANGUAGE, I-PERSON, S-EVENT, S-DATE, B-QUANTITY, E-QUANTITY, S-TIME, B-TIME, I-TIME, E-TIME, B-GPE, E-GPE, S-ORG, I-GPE, S-NORP, B-FAC, I-FAC, E-FAC, B-NORP, E-NORP, S-PERCENT, B-ORG, E-ORG, B-LANGUAGE, E-LANGUAGE, I-CARDINAL, I-ORG, S-WORK_OF_ART, I-QUANTITY, B-MONEY


In [17]:
len(D)

114

In [18]:
ent=[]
label=[]
for i in D :
    ent.append(i)
    label.append(D[i])

In [19]:
final1=pd.DataFrame({"entities":ent, "label":label})

In [20]:
P=[]
OR=[]
GP=[]
Date=[]
LOC=[]
for i in range(114):
    if final1.label.loc[i]=="PERSON":
        P.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="ORG":
        OR.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="GPE":
        GP.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="DATE":
        Date.append(final1.entities.loc[i])
    elif final1.label.loc[i]=="LOC":
        LOC.append(final1.entities.loc[i])
        

In [21]:
len(Date)

45

In [22]:
Date

['May 29th 1865',
 '27th day of Novb 1865',
 '27th day of Novbr 1865',
 '63 year',
 '28 day of Sept 1865',
 'May 47 PM',
 '39 year',
 'Maj 47th',
 '31 day of Oct 1865',
 '61 year',
 '31st day of August AD 1865',
 'August 24 1865',
 '3 day of August 1865',
 'May 4',
 '3 day of Augt 1865',
 'Maj 47',
 '48 year',
 '16th day of Sept 1865',
 '58 year',
 '16 day of October 1865',
 'twenty eight year',
 'July 22',
 '6 day of Nov 1865',
 '58 year of age',
 'Sept 29 1865',
 '29th day of September 1865',
 '24 year',
 'Sept 29th 1865',
 '29th day of Sept 29 1865',
 'Maj 66th',
 '3d day of February 1868',
 '7th day of September 186',
 '45years',
 'Aug 20 1865',
 '20th day of August 1865',
 'Thirty nine',
 'Aug 20th 1865',
 'August 29th 1865',
 '22nd day of August 1865',
 '18th Sept 1865',
 'Aug 2 7 66',
 'Aug 31 1865',
 '31st day of Aug 1865',
 '26 Years',
 '31 day of Aug 1865']

In [23]:
len(P)

43

In [24]:
P

['Anne H Elliott',
 'Andrew Johnson',
 'S Wilde Durson',
 'Hazel',
 'I E C Whaley',
 'Levi Steuben',
 'E C Whaley',
 'E Montgomery',
 'ANDREW JOHNSON',
 'Levi Stuber Major',
 'Montgomery',
 'Hu Kennedy',
 'JB Hiserodt',
 'Whaley',
 'William Whaley Sworn',
 'Levi Stuber',
 'I J F Steinmeyer',
 'JF Steinmeyer',
 'Joseph Edings',
 'Frank Geise',
 'Vicksburg',
 'Jacob Schroder',
 'Jacob Schroder Sworn',
 'JW Baxter',
 'J H Mathews',
 'Frank E Miller',
 'Bergevin',
 'P H Joor',
 'Frank Conner',
 'Caroline Belitzer',
 'caroline Belitzer',
 'Hiram Smith',
 'mark Sworn',
 'Geo D Carpenter',
 'Aiken Burnwell',
 'James E Thuring',
 'J E Thuring',
 'Robert A Wilson',
 'James D Kendall',
 'Oath',
 'I H F Holwell',
 'H F Holwell',
 'Mathews']

****xlm-roberta-base-finetuned-panx-ner****
for Person

In [25]:
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

tokenizer = AutoTokenizer.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")
model = AutoModelForTokenClassification.from_pretrained("gunghio/xlm-roberta-base-finetuned-panx-ner")

nlp1 = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="first")

Downloading:   0%|          | 0.00/398 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.83M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/16.3M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/239 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/964 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.03G [00:00<?, ?B/s]

In [26]:
T7=[]
example = text

for i in range(20):
        ner_results = nlp1(example[i])
        T7.append(ner_results)

In [27]:
T7[1][1]

{'entity_group': 'ORG',
 'score': 0.87107897,
 'word': 'AmnestyProclamation',
 'start': 45,
 'end': 65}

In [28]:
Per2=[]
ORg2=[]
GPe2=[]
for i in range(20):
    for j in range(len(T7[i])):
        if ( T7[i][j]["entity_group"]=='ORG') & ((T7[i][j]["score"]) > 0.90):
            ORg2.append(T7[i][j]["word"])
        elif ( T7[i][j]["entity_group"]=='PER')  & ((T7[i][j]["score"]) > 0.85):
            Per2.append(T7[i][j]["word"])
        elif ( T7[i][j]["entity_group"]=='LOC') & ((T7[i][j]["score"]) > 0.84) :
            GPe2.append(T7[i][j]["word"])
GPe2=np.array(GPe2)
ORg2=np.array(ORg2)
Per2=np.array(Per2)



**best result**

In [29]:
np.unique(Per2)

array(['ANDREWJOHNSON', 'AndrewJohnson', 'AnneHElliott',
       'CarolineBelitzer', 'FrankConner', 'HiramSmith', 'JFSteinmeyer',
       'JacobSchroder', 'LeviSteuben', 'RobertAWilson', 'WildeDurson'],
      dtype='<U16')

In [30]:
np.unique(Person)

array(['Amnesty Oath', 'Amnesty Oath Administered At', 'Andrew Johnson',
       'Anne H Elliott', 'Anne H Elliott United States of America',
       'Bolt Capt 54th', 'Brown', 'Captn', 'Captn Commanding TRIPLICATE',
       'Charleston Sc', 'City Miss', 'Copy F K Field Capt', 'County',
       'Frank Conner', 'Frank E Miller', 'Gibbs Henderson', 'Gry',
       'Hazel', 'Hiram Smith', 'Hiserodt', 'Hu Kennedy',
       'JF Steinmeyer Sworn', 'Jacob Schroder', 'Jacob Schroder Sworn',
       'James D Kendall Amnesty Oath Administered At Oacen Springs',
       'Joseph Edings', 'Laborer', 'Oaths', 'Pass Christian',
       'Retained Copy', 'Robert A', 'Strikethrough',
       'Thomas Andrew Oath of', 'Triplicate AMNESTY', 'Walhalla',
       'Whaley', 'William Whaley Sworn', 'Woodville Wilkinson'],
      dtype='<U58')

In [31]:
np.unique(ORG)

array(['AHE', 'AMNESTY', 'ANDREW', 'Amnesty', 'Amnesty Proclamation',
       'Belitzer Sworn', 'Bray', 'Charleston State of So Carolina',
       'Color of Eyes Blue Color of Hair Dark Characteristics',
       'Department of State', 'Emancipation of Slaves',
       'Eyes Blue Color of Hair', 'H F Holwell of', 'H Mathews Capt',
       'INTERREVENUE', 'JF Steinmeyer Sworn', 'Joseph Edings Sworn',
       'Levi Stuber', 'Magistrate S Wilde Durson AHE AHE AHE',
       'OATH I P P Bergevin',
       'P H Joor Justice of Peace Wilkinson County State of',
       'Rebellion reference Emancipation of Slaves',
       'Regt U S C Infy Provost Marshal', 'Sgd Montgomery Sworn', 'Sworn',
       'UNITED STATES FORCES No 1282 PROVOST MARSHALS OFFICE',
       'US Col Infty Provost Marshal', 'USCI Commanding DUPLICATE',
       'USCI Commanding TRIPLICATE', 'United State of America',
       'United States Union', 'United States Union of State',
       'Whaley Sworn', 'Wilson'], dtype='<U53')

In [32]:
np.unique(GP)

array(['Charleston', 'Charleston S C', 'Charleston SC', 'Charleston Sc',
       'City of Charleston', 'Mississippi City', 'So Carolina',
       'South Carolina', 'UNITED STATES OF AMERICA', 'US',
       'United States', 'United States of America',
       'Walhalla State of South Carolina', 'strikethrough County'],
      dtype='<U32')

In [33]:
np.unique(ORg2)

array(['AikenSC', 'AlmightyGod', 'AmnestyOath', 'CharlestonSC',
       'ConstitutionofUnitedStatesUnionofStates', 'EmancipationofSlaves',
       'IWmWhaley', 'ThosGibbs', 'VicksburgMiss', 'WAFleitasAmnestyOath'],
      dtype='<U39')

In [34]:
np.unique(Date)

array(['16 day of October 1865', '16th day of Sept 1865',
       '18th Sept 1865', '20th day of August 1865',
       '22nd day of August 1865', '24 year', '26 Years',
       '27th day of Novb 1865', '27th day of Novbr 1865',
       '28 day of Sept 1865', '29th day of Sept 29 1865',
       '29th day of September 1865', '3 day of Augt 1865',
       '3 day of August 1865', '31 day of Aug 1865', '31 day of Oct 1865',
       '31st day of Aug 1865', '31st day of August AD 1865', '39 year',
       '3d day of February 1868', '45years', '48 year', '58 year',
       '58 year of age', '6 day of Nov 1865', '61 year', '63 year',
       '7th day of September 186', 'Aug 2 7 66', 'Aug 20 1865',
       'Aug 20th 1865', 'Aug 31 1865', 'August 24 1865',
       'August 29th 1865', 'July 22', 'Maj 47', 'Maj 47th', 'Maj 66th',
       'May 29th 1865', 'May 4', 'May 47 PM', 'Sept 29 1865',
       'Sept 29th 1865', 'Thirty nine', 'twenty eight year'], dtype='<U26')

****